# Advanced programming: assignment 3

### Daniel A.
### UID: 100444499

#### Importing Libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, PredefinedSplit
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.utils.validation import check_is_fitted

#### Importing and Manipulating the data

In [3]:
# importing the data
data = pd.read_pickle("./data/wind_pickle.pickle")

# dropping cols
data.drop(['steps', 'month', 'day', 'hour'], axis=1, inplace=True)

# set seed
my_NIA = 34291182
np.random.seed(my_NIA)

# selecting 10% of cols except year and energy
cols = list(set(data.columns) - {'year', 'energy'})
cols_selected = [np.random.choice(cols) for x in range(int(len(cols)*0.1))]

# adding 5% missing values at random places
for col in cols_selected:
    selected_indexes = [np.random.choice(
        data.index) for x in range(int(len(data)*0.05))]
    for idx in selected_indexes:
        data.loc[idx,col] = np.nan

# saving the dataset
data.to_pickle('./data/data.pickle')

#### Further data preprocessing for modelling

In [4]:
# train partition
train = data[data['year'].isin([2005,2006])].drop('year',axis=1)
X_train = train[[x for x in train.columns if x != 'energy']].values
y_train = train['energy'].values
# validation partition
validation = data[data['year'].isin([2007,2008])].drop('year',axis=1)
X_validation = validation[[x for x in train.columns if x != 'energy']].values
y_validation = validation['energy'].values
# test partition
test = data[data['year'].isin([2009,2010])].drop('year',axis=1)
X_test = test[[x for x in train.columns if x != 'energy']].values
y_test = test['energy'].values

### 1 - Model selection and hyper-parameter tuning

#### Training and evaluating KNN, Reg trees, SVMs with default hyper-parameters

In [5]:
# creating a dataframe to keep track of scores and results
scores = {'knn':[],'svm':[],'dtr':[]}

Here we define 3 pipelines, one for KNN, another one for SVM and another one for Decision Trees. 

For KNN and SVM we use imputation (with simple imputer using the mean strategy) and a min max scaler.

For Decision trees we only perform imputation and then the model fit.

We create a dictionary with each model name as key and each pipeline as value.

We then create a loop where we print the name of the model (to know how far the loop has gone) and then we append to the scores dataframe the MAE of each prediction.

In [6]:
# knn pipeline
knn = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler()),
    ('model', KNeighborsRegressor())])

# svm
svm = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler()),
    ('model', SVR())])

# tree
dtr = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('model', DecisionTreeRegressor())])

# models
models = {'knn':knn, 'svm':svm, 'dtr':dtr}
for name, model in models.items():
    print(name)
    models[name].fit(X_train, y_train)
    scores[name].append(mean_absolute_error(y_test, models[name].predict(X_test)))

knn
svm
dtr


#### Training and evaluating KNN, Reg trees, SVMs with imputation and hyper-parameter tuning using RandomizedSearch

We define a predefined split

In [7]:
val_indeces = np.zeros(X_validation.shape[0])
val_indeces[:round(2/3*X_validation.shape[0])] = -1
pdfsplit = PredefinedSplit(val_indeces)

In this section we define pipelines with the intention to use RandomizedSearch to perform hyperparameter tuning.

For KNN we first define the parameters for steps in the pipeline:

For the model we test:
 - n_neighbors for all values between 2 and 30 with steps of 1
 - leaf_size for all values between 28 and 40 with steps of 1
 - algorithm for auto, ball_tree, kd_tree and brute
 - weights for uniform and distance

For the imputer we test:
 - strategy for mean and median

After defining the hyper-parameters to test we define the pipeline with the same steps as before, first imputer, then the scaler and then the model. Then we define the RandomizedSearchCV hyper-parameter tuning with 10 iterations and using MAE for scoring.

In [8]:
# knn
knn_params = {
    'model__n_neighbors':np.arange(2,30,1),
    'model__leaf_size':np.arange(28,40,1),
    'model__algorithm':['auto','ball_tree','kd_tree','brute'],
    'model__weights':['uniform','distance'],
    'impute__strategy':['mean','median']
}

knn = Pipeline([
    ('impute', SimpleImputer()),
    ('scaler', MinMaxScaler()),
    ('model', KNeighborsRegressor())])

knn_RS = RandomizedSearchCV(estimator=knn,
                            param_distributions=knn_params,
                            n_iter=10,
                            cv=pdfsplit,
                            scoring='neg_mean_absolute_error')

For SVM we first define the parameters for steps in the pipeline:

For the model we test:
 - degree for all values between 2 and 7 with steps of 1
 - gamma for all values between 0.0001 and 0.1 with steps of 0.005 along with the scale and auto options
 - shrinking with True and False
 - C for all values between 0 and 5 with steps of 0.5

For the imputer we test:
 - strategy for mean and median

After defining the hyper-parameters to test we define the pipeline with the same steps as before, first imputer, then the scaler and then the model. Then we define the RandomizedSearchCV hyper-parameter tuning with 10 iterations and using MAE for scoring.

In [9]:
# svm
svm_params = {
    'model__degree':np.arange(2,7,1),
    'model__gamma':list({'scale','auto'}.union(set(np.arange(0.0001,0.1,0.005)))),
    'model__shrinking':[True, False],
    'model__C':np.arange(0,5,0.5),
    'impute__strategy':['mean','median']
}

svm = Pipeline([
    ('impute', SimpleImputer()),
    ('scaler', MinMaxScaler()),
    ('model', SVR())])


svm_RS = RandomizedSearchCV(estimator=svm,
                            param_distributions=svm_params,
                            n_iter=10,
                            cv=pdfsplit,
                            scoring='neg_mean_absolute_error')

For Decision trees we first define the parameters for steps in the pipeline:

For the model we test:
 - criterion for mse, friedman_mse, mae and poisson
 - splitter for best and random
 - max_dept for all values between 2 and 14 with steps of 2
 - min_samples_split for all values between 2 and 10 with steps of 2
 - min_samples_leaf for all values between 1 and 10 with steps of 1
 - max_features for auto, sqrt and log2

For the imputer we test:
 - strategy for mean and median

After defining the hyper-parameters to test we define the pipeline with the same steps as before, first imputer, then the scaler and then the model. Then we define the RandomizedSearchCV hyper-parameter tuning with 10 iterations and using MAE for scoring.

In [10]:
# tree
dtr_params = {
    'model__criterion':['mse', 'friedman_mse', 'mae', 'poisson'],
    'model__splitter':['best','random'],
    'model__max_depth':np.arange(2,14,2),
    'model__min_samples_split':np.arange(2,10,2),
    'model__min_samples_leaf':np.arange(1,10,1),
    'model__max_features':['auto','sqrt','log2'],
    'impute__strategy':['mean','median'],
}

dtr = Pipeline([
    ('impute', SimpleImputer()),
    ('model', DecisionTreeRegressor())])

dtr_RS = RandomizedSearchCV(estimator=dtr,
                            param_distributions=dtr_params,
                            n_iter=10,
                            cv=pdfsplit,
                            scoring='neg_mean_absolute_error')

Like before, we loop over the model names (keys of the models dictionary) and the RandomizedSearchCV corresponding to each model's pipeline (values of the models dictionary). We fit using the RandomizedSearchCV and add the mean absolute error to the scores dictionary.

In [11]:
# models
models = {'knn':knn_RS, 'svm':svm_RS, 'dtr':dtr_RS}

# running fit
for name, model in models.items():
    print(name)
    models[name].fit(X_train, y_train)
    scores[name].append(mean_absolute_error(y_test, models[name].predict(X_test)))

knn
svm
dtr


#### Checking scores for the 6 models

Row 0 are the models with default params and Row 1 are the models with hyperparameter tuning done

In [12]:
pd.DataFrame(scores)

,knn,svm,dtr
0,363.636970,517.567987,393.728190
1,366.139236,469.443647,323.745465


Our models with hyper-parameter tuning yield better results with out model with the lowest MAE (323.745465) being the decision tree with hyper-parameter tuning done.

According to the MAE,0 the best model is the decision trees with hyperparameter tuning done

### 2 - Attribute selection

#### First pipeline: SelectKBest included in the pipeline

For the first Pipeline we define the parameters for steps in the pipeline:

For the model we test:
 - n_neighbors between 2 and 50 with steps of 1
  
For the feature selection (SelectKBest):
 - k between 2 and 40 with steps of 1 and the 'all' parameter (which just tries all features)

For the imputer we test:
 - strategy for mean and median

After defining the hyper-parameters to test we define the pipeline with the same steps as before, first imputer, then the scaler and then the model. Then we define the RandomizedSearchCV hyper-parameter tuning with 25 iterations and using MAE for scoring.

In [13]:
# knn
knn_params_1 = {
    'model__n_neighbors':np.arange(2,50,1),
    'featsel__k':list(set(np.arange(2,40,1)).union({'all'})),
    'impute__strategy':['mean','median']
}

knn_1 = Pipeline([
    ('impute', SimpleImputer()),
    ('scaler', MinMaxScaler()),
    ('featsel',SelectKBest()),
    ('model', KNeighborsRegressor())])

knn_RS_1 = RandomizedSearchCV(estimator=knn_1,
                            param_distributions=knn_params_1,
                            n_iter=25,
                            cv=pdfsplit,
                            scoring='neg_mean_absolute_error')

#### Second pipeline: PCA included in the pipeline

For the second Pipeline we define the parameters for steps in the pipeline:

For the model we test:
 - n_neighbors between 2 and 50 with steps of 1
  
For the variable decomposition (PCA):
 - k between 2 and 40 with steps of 1 and the 'all' parameter (which just tries all features)

For the imputer we test:
 - strategy for mean and median

After defining the hyper-parameters to test we define the pipeline with the same steps as before, first imputer, then the scaler and then the model. Then we define the RandomizedSearchCV hyper-parameter tuning with 10 iterations and using MAE for scoring.

In [14]:
# knn
knn_params_2 = {
    'model__n_neighbors':np.arange(2,50,1),
    'decomp__n_components':list(set(np.arange(2,40,1)).union({'all','mle'})),
    'impute__strategy':['mean','median']
}

knn_2 = Pipeline([
    ('impute', SimpleImputer()),
    ('scaler', MinMaxScaler()),
    ('decomp',PCA()),
    ('model', KNeighborsRegressor())])

knn_RS_2 = RandomizedSearchCV(estimator=knn_2,
                            param_distributions=knn_params_2,
                            n_iter=10,
                            cv=pdfsplit,
                            scoring='neg_mean_absolute_error')

#### Third pipeline: both PCA and SelectKBest included in the pipeline (running SelectKBest first)

For the third Pipeline we define the parameters for steps in the pipeline:

For the model we test:
 - n_neighbors between 2 and 50 with steps of 1
  
For the feature selection (SelectKBest):
 - k between 2 and 40 with steps of 1 and the 'all' parameter (which just tries all features)

For the variable decomposition (PCA):
 - k between 2 and 40 with steps of 1 and the 'all' parameter (which just tries all features) and 'mle'

For the imputer we test:
 - strategy for mean and median

After defining the hyper-parameters to test we define the pipeline with the same steps as before, first imputer, then the scaler and then the model. Then we define the RandomizedSearchCV hyper-parameter tuning with 10 iterations and using MAE for scoring.

For this pipeline we run the feature selection before the variable decomposition.

In [15]:
# knn
knn_params_3 = {
    'model__n_neighbors':np.arange(2,50,1),
    'featsel__k':list(set(np.arange(2,40,1)).union({'all'})),
    'decomp__n_components':list(set(np.arange(2,40,1)).union({'all','mle'})),
    'impute__strategy':['mean','median']
}

knn_3 = Pipeline([
    ('impute', SimpleImputer()),
    ('scaler', MinMaxScaler()),
    ('featsel',SelectKBest()),
    ('decomp',PCA()),
    ('model', KNeighborsRegressor())])

knn_RS_3 = RandomizedSearchCV(estimator=knn_3,
                            param_distributions=knn_params_3,
                            n_iter=10,
                            cv=pdfsplit,
                            scoring='neg_mean_absolute_error')

#### Fourth pipeline: both PCA and SelectKBest included in the pipeline (running PCA first)

For the fourth Pipeline we define the parameters for steps in the pipeline:

For the model we test:
 - n_neighbors between 2 and 50 with steps of 1
  
For the feature selection (SelectKBest):
 - k between 2 and 40 with steps of 1 and the 'all' parameter (which just tries all features)

For the variable decomposition (PCA):
 - k between 2 and 40 with steps of 1 and the 'all' parameter (which just tries all features) and 'mle'

For the imputer we test:
 - strategy for mean and median

After defining the hyper-parameters to test we define the pipeline with the same steps as before, first imputer, then the scaler and then the model. Then we define the RandomizedSearchCV hyper-parameter tuning with 10 iterations and using MAE for scoring.

For this pipeline we run the the variable decomposition before the feature selection.

In [16]:
# knn
knn_params_4 = {
    'model__n_neighbors':np.arange(2,50,1),
    'decomp__n_components':list(set(np.arange(2,40,1)).union({'all','mle'})),
    'featsel__k':list(set(np.arange(2,40,1)).union({'all'})),
    'impute__strategy':['mean','median']
}

knn_4 = Pipeline([
    ('impute', SimpleImputer()),
    ('scaler', MinMaxScaler()),
    ('decomp',PCA()),
    ('featsel',SelectKBest()),
    ('model', KNeighborsRegressor())])

knn_RS_4 = RandomizedSearchCV(estimator=knn_4,
                            param_distributions=knn_params_4,
                            n_iter=10,
                            cv=pdfsplit,
                            scoring='neg_mean_absolute_error')

We run the models in the same manner as before, we have 4 instances of RandomizedSearchCV and we append the MAE of the models to the datafram

In [17]:
# creating a dataframe to keep track of scores and results
scores = {'pipeline':[],'knn':[]}

# models
models = {'knn_1':knn_RS_1, 'knn_2':knn_RS_2, 'knn_3':knn_RS_3, 'knn_4':knn_RS_4}

# running fit
for idx, (name, model) in enumerate(models.items()):
    print(name)
    models[name].fit(X_train, y_train)
    scores['pipeline'].append(idx + 1)
    scores['knn'].append(mean_absolute_error(y_test, models[name].predict(X_test)))

knn_1
knn_2
knn_3
knn_4


And we see our scores below:

In [19]:
pd.DataFrame(scores)

,pipeline,knn
0,1,376.815078
1,2,361.083188
2,3,367.272739
3,4,419.986353


We can notice that the best pipeline is the second one we tried, it gets the lowest MAE, closely followed by the first pipeline, however, it does not improve on the previous decision tree regressor result. 

We could therefore say that tuning a decision tree regressor might be more worthwhile than tuning a KNN model extensively.

#### Extracting best features from the first pipeline

We can see from our first pipeline the selected columns are as follows:

In [29]:
sup = models['knn_1'].best_estimator_.named_steps['featsel'].get_support(indices=True)
feats = data.iloc[:,sup]
feats.columns

Index(['iews.1', 'iews.2', 'iews.3', 'iews.7', 'iews.8', 'iews.12', 'iews.13',
       'iews.17', 'iews.18', 'iews.24', 'iews.25', 'inss.1', 'inss.2',
       'inss.3', 'inss.4', 'inss.5', 'inss.6', 'inss.7', 'inss.8', 'inss.9',
       'inss.10', 'inss.11', 'inss.12', 'inss.13', 'inss.14', 'inss.15',
       'inss.16', 'inss.17', 'inss.18', 'inss.19', 'inss.20', 'inss.21',
       'inss.22', 'inss.23'],
      dtype='object')

In [30]:
len(feats.columns)

34

We can see that among the most common locations selected by the model (using k=34 as selected by the hyper parameter tuning), the sotavento location is present twice (iews.13 and inss.13), there's also other locations present twice, as the tuning only chooses Instantaneous eastward turbulent surface stress and Instantaneous northward turbulent surface, for these 2 types of metrics, the most common locations are:
 - 1
 - 2
 - 3
 - 7
 - 8
 - 12
 - 13
 - 17
 - 18